In [124]:
import pandas as pd
import geopandas as gpd
import psrcelmerpy

In [125]:
# Load TransRefEdges geodata
eg_conn = psrcelmerpy.ElmerGeoConn()
gdf = eg_conn.read_geolayer('transrefedges')

# Load Mode Attributes table
df_mode_attr = eg_conn.get_table(schema='dbo', table_name='modeattributes')

In [126]:
# Merge mode attributes to TransRefEdges
gdf = gdf.merge(df_mode_attr, on='PSRCEdgeID', how='left')

In [134]:
# Load bike stress table, adapted from Lowry
df_data = pd.read_csv('bike_stress_table.csv')

In [133]:
# FIXME: transRefEdges lanes are directional and Lowry data is bi-directional
# 

In [106]:
# Reformat speed limit and lanes to match the bike stress data
# Creating a new column for speed limits where minimum is capped at 25 and max at 35
gdf['IJSpeedLimit_new'] = gdf['IJSpeedLimit'].copy()
gdf.loc[gdf['IJSpeedLimit'] <= 25, 'IJSpeedLimit_new'] = 25
gdf.loc[gdf['IJSpeedLimit'] >= 35, 'IJSpeedLimit_new'] = 35

# Set maximum lanes at 6 and minimum at 2

gdf['IJLanesGPAM_new'] = gdf['IJLanesGPAM'].copy().astype('int')
gdf.loc[gdf['IJLanesGPAM'] >= 6, 'IJLanesGPAM_new'] = 6
gdf.loc[gdf['IJLanesGPAM'] <2, 'IJLanesGPAM_new'] = 2

# Lowry has different values for neighborhood street for each bike facility type
# Create a flag for an local street
gdf['LocalStreet'] = 0
gdf.loc[gdf['FacilityType'] == 9, 'LocalStreet'] = 1
# gdf['FacilityType'].unique()

# If no bike types assign 0
gdf.loc[gdf['IJBikeType'].isnull(), 'IJBikeType'] = 0

gdf['IJBikeType'] = gdf['IJBikeType'].astype('int')

array(['NoBikeFacility', 'Sharrows', 'BikeLane', 'BufferedBikeLane',
       'ProtectedBikeLane', 'SharedUsePath'], dtype=object)

In [118]:
# Define transRefEdges facility types to match the study definitions

bike_type_map = {
    0: 'NoBikeFacility',    # No Bike Lane
    1: 'BikeLane',    # Striped Bike Lane
    2: 'ProtectedBikeLane',    # Protected Bike Lane
    3: 'NoBikeFacility',    # Paved Shoulder
    4: 'Sharrows',    # Shared Lane Markings
    5: 'NoBikeFacility',    # Bike Provision Undefined
    6: 'NoBikeFacility',    # Bike Provision Undefined
    8: 'SharedUsePath',    # Shared Use Path
    9: 'BufferedBikeLane',    # Buffered Bike Lane 
    10: 'NoBikeFacility',     # Neighborhood Greenway
    
}

# Create a mapping of local vs other area type
area_type_map = {
    0: 'Other',
    1: 'Neighborhood'
}

# Create new fields that use the bike facility and area type mapping
gdf['IJ_BikeFacility'] = gdf['IJBikeType'].map(bike_type_map)
gdf['IJAreaType'] = gdf['LocalStreet'].map(area_type_map)

# We will follow this designation for any local street with speeds 30 mph or less and 2 lanes (1 in each direction)
# If speed limit > 30, set area type to non-neighborhood
gdf.loc[(gdf['IJSpeedLimit']>30, 'IJAreaType')] = 'Other'

# If number of lanes > 2, set area type to non-neighborhood
# FIXME: should probably just be 1 lane in each direction
gdf.loc[(gdf['IJLanesGPAM']>2, 'IJAreaType')] = 'Other'

# Merge to df_data to get stress values
gdf = gdf.merge(df_data, left_on=['IJSpeedLimit_new', 'IJLanesGPAM_new', 'IJ_BikeFacility','IJAreaType'], 
          right_on=['SpeedLimit', 'Lanes', 'BikeFacility', 'AreaType'], how='left')

In [120]:
# Assign LTS based on stress factor
gdf.loc[gdf['Factor'] < 0.1, 'LTS'] = 1
gdf.loc[(gdf['Factor'] >= 0.1) & (gdf['Factor'] < 0.3), 'LTS'] = 2
gdf.loc[(gdf['Factor'] >= 0.3) & (gdf['Factor'] < 0.6), 'LTS'] = 3
gdf.loc[gdf['Factor'] >= 0.6, 'LTS'] = 4

In [ ]:
# FIXME: add slope 